In [4]:
from kaggle.api.kaggle_api_extended import KaggleApi
import os

# Autenticação
api = KaggleApi()
api.authenticate()

In [ ]:
DATA_DIR = 'data'
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

api.dataset_download_files('diraizel/anime-images-dataset', path=DATA_DIR, unzip=True)

Dataset URL: https://www.kaggle.com/datasets/diraizel/anime-images-dataset


In [14]:
# Ajuste o DATA_DIR para apontar pra pasta certa
DATA_DIR   = 'data/data/anime_images'
TARGET_DIR = 'data/10_animes'

import shutil, os
os.makedirs(TARGET_DIR, exist_ok=True)

# Liste todas as pastas de anime
all_animes = sorted(os.listdir(DATA_DIR))
print(all_animes)  # agora devem aparecer ~200 nomes

['A Lull in the Sea', 'A Place Further Than The Universe', 'A Silent Voice', 'AKIRA', 'Ace of Diamond', 'Akatsuki no Yona', 'Angel Beats!', 'Aria The Natural', 'Aria The Origination', 'Assassination Classroom', 'Attack on Titan', 'Baccano!', 'Bakemonogatari', 'Bakuman', 'Barakamon', 'Beck Mongolian Chop Squad', 'Berserk', 'Big Windup!', 'Black Butler', 'Black Lagoon', 'Bunny Drop', 'Cardcaptor Sakura', 'Carpcaptor Sakura', 'Castle in the Sky', 'Chihayafuru', 'Clannad', 'Code Geass', 'Cross Game', 'D.Gray-man', 'DARLING in the FRANXX', 'Daily Lives of High School Boys', 'Darker than Black', 'Death Note', 'Death Parade', 'Den-noh Coil', 'Descending Stories Showa Genroku Rakugo Shinju', 'Detective Conan', 'Detroit Metal City The Animated Series', 'Dragon Ball', 'Dragon Ball Z', 'Durarara', 'Durarara!!', 'ERASED', 'Erin', 'Eureka Seven', 'FateZero', 'Fatestay night', 'Fighting Spirit Special', 'From the New World', 'Full Metal Panic', 'Fullmetal Alchemist Brotherhood', 'Gankutsuou The Coun

In [15]:
# Ajuste manual dos animes que você quer usar
selected = [
    'AKIRA',
    'Attack on Titan',
    'Cardcaptor Sakura',
    'Fullmetal Alchemist Brotherhood',
    'Dragon Ball Z',
    'Death Note',
    'Haikyu!!',
    'My Hero Academia',
    'Hunter x Hunter',
    'Nana',
    'Nanatsu no Taizai',
    'Naruto',
    'Yu Yu Hakusho',
    'Your Name',
    'Violet Evergarden',
    'One Piece',
]

# Aponte para onde estão de fato as ~200 pastas
DATA_DIR   = 'data/data/anime_images'
TARGET_DIR = 'data/16_animes'  # ou renomeie para 'data/16_animes'

os.makedirs(TARGET_DIR, exist_ok=True)

for anime in selected:
    src = os.path.join(DATA_DIR, anime)
    dst = os.path.join(TARGET_DIR, anime)
    shutil.copytree(src, dst)


In [1]:
import tensorflow as tf
print(tf.__version__)                      # deve responder algo como '2.x.x-dev…'
from tensorflow.keras.preprocessing.image import ImageDataGenerator


ImportError: Traceback (most recent call last):
  File "c:\Users\mrsilva\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: Uma rotina de inicialização da biblioteca de vínculo dinâmico (DLL) falhou.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [22]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (128, 128)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

train_gen = train_datagen.flow_from_directory(
    'data/16_animes',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_gen = train_datagen.flow_from_directory(
    'data/16_animes',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

ImportError: Traceback (most recent call last):
  File "c:\Users\mrsilva\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: Uma rotina de inicialização da biblioteca de vínculo dinâmico (DLL) falhou.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.